# 顔認証で集合写真から人数を計算

In [ ]:
%pip install ultralytics dill opencv-python matplotlib facenet-pytorch mtcnn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# まずYOLOを使ってみました

In [ ]:
import cv2
from ultralytics import YOLO

# 1. モデルのロード (YOLOv8nを使用)
model = YOLO('yolov8n.pt')

# 2. 検出対象のクラスIDを定義
# COCOデータセットでは 'person' のIDは 0 です
person_class_id = 0

# 3. 画像のパス
image_path = 'IMG_5328.JPG'

# 4. 画像のロード
try:
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError
except FileNotFoundError:
    print(f"エラー: 画像ファイルが見つかりません: {image_path}")
    exit()

# 5. 検出の実行
# conf=0.5: 信頼度0.5以上の結果のみを使用
results = model(img, conf=0.5)

# 6. 人の検出結果をフィルタリングし、カウント
person_count = 0
if results and results[0].boxes:
    for box in results[0].boxes:
        # クラスIDが 'person' であるかを確認
        if int(box.cls[0]) == person_class_id:
            person_count += 1
            
            # (オプション) 検出された人の周りにバウンディングボックスを描画
            x1, y1, x2, y2 = [int(val) for val in box.xyxy[0]]
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # (オプション) ラベルと信頼度を表示
            confidence = box.conf[0].item()
            label = f'Person: {confidence:.2f}'
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 7. 結果の表示
print(f"画像 '{image_path}' で検出された人数: {person_count}人")

# (オプション) 結果画像を表示
cv2.putText(img, f'Total People: {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
cv2.imshow("YOLO Person Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 480x640 9 persons, 141.6ms
Speed: 40.3ms preprocess, 141.6ms inference, 19.3ms postprocess per image at shape (1, 3, 480, 640)
画像 'IMG_5328.JPG' で検出された人数: 9人


# 次にMTCNNを使ってみました

In [ ]:
import cv2
from facenet_pytorch import MTCNN
import torch

# 1. モデルのロード
# デバイスの設定 (GPUが利用可能ならGPUを使用)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# min_face_size: 検出する顔の最小サイズ (ピクセル)
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

# 2. 画像のパス
image_path = 'IMG_5328.JPG'

# 3. 画像のロード
try:
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError
except FileNotFoundError:
    print(f"エラー: 画像ファイルが見つかりません: {image_path}")
    exit()

# OpenCVの画像をPIL形式に変換 (MTCNNの多くの実装がPIL入力を要求するため)
from PIL import Image
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# 4. 顔の検出
# boxes: 検出された顔のバウンディングボックス
# probs: 検出された顔の信頼度
boxes, probs = mtcnn.detect(img_pil)

face_count = 0
if boxes is not None:
    face_count = len(boxes)
    # 5. 検出結果の描画
    for box in boxes:
        # バウンディングボックスの座標を取得
        x1, y1, x2, y2 = [int(val) for val in box]
        
        # バウンディングボックスを描画
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        # (オプション) 信頼度を表示 (probsがNoneでないことを確認)
        if probs is not None:
             label = f'Face: {probs[face_count-1]:.2f}'
             cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
# 6. 結果の表示
print(f"画像 '{image_path}' で検出された顔の数: {face_count}個")

# (オプション) 結果画像を表示
cv2.putText(img, f'Total Faces: {face_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
cv2.imshow("MTCNN Face Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

画像 'IMG_5328.JPG' で検出された顔の数: 3個


# まとめ
今回はYOLOとMTCNNの両方を使って人数計算をしてみました。

YOLOは9人と計算され、MTCNNは３人と計算されて、本当は１２人ですが、うまく計算しきれなかったです。

しかし、奇跡的にYOLOとMTCNNの人数を足すとちょうど１２人になっていて驚きました。

やっぱり良い写真を選んだと思うんですが反応しにくいらしかったです。

結果としてはMTCNNよりYOLOの方がよかったと思いました。